# Example notebook for inverse folding model

## Load model

In [1]:
from esm.pretrained import esm_if1_gvp4_t16_142M_UR50
model, alphabet = esm_if1_gvp4_t16_142M_UR50()

/home/chloehsu/inverse_opensource/esm_fork/esm/pretrained.py:172: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


## Load structure from PDB or CIF files

As an example, let's look at Golgi Casein Kinase, the [PDB Molecule of the Month from January 2022](https://pdb101.rcsb.org/motm/265).

Milk is a complex mixture of proteins, fats, and nutrients that provides everything that a growing infant needs. Most of the protein in cow’s milk is casein, whereas human milk has lesser amounts of casein. 

The Golgi casein kinase (PDB entry 5YH2) adds phosphates to casein and also to many other types of secreted proteins. It is most active as a complex of two similar types of proteins. Fam20C is the catalytic subunit. It binds to casein and transfers a phosphate from ATP to the protein.

In this example, let's focus on chain C (the catalytic subunit Fam20C).

In [2]:
from inverse_folding import load_structure, extract_coords_from_structure
fpath = 'example/5YH2.cif' # .pdb format is also acceptable
chain_id = 'C'
structure = load_structure(fpath, chain_id)
coords, native_seq = extract_coords_from_structure(structure)
print('Native sequence:')
print(native_seq)

Found 6 chains: ['A' 'C' 'B' 'D' 'A' 'B'] 

Loaded chain C

Native sequence:
SVLQSLFEHPLYRTVLPDLTEEDTLFNLNAEIRLYPKAASESYPNWLRFHIGINRYELYSRHNPVIAALLRDLLSQKISSVGMKSGGTQLKLIMSFQNYGQALFKPMKQTREQETPPDFFYFSDFERHNAEIAAFHLDRILDFRRVPPVAGRLVNMTREIRDVTRDKKLWRTFFVSPANNICFYGECSYYCSTEHALCGKPDQIEGSLAAFLPDLALAKRKTWRNPWRRSYHKRKKAEWEVDPDYCDEVKQTPPYDRGTRLLDIMDMTIFDFLMGNMDRHHYETFEKFGNDTFIIHLDNGRGFGKHSHDEMSILVPLTQCCRVKRSTYLRLQLLAKEEYKLSSLMEESLLQDRLVPVLIKPHLEALDRRLRLVLKVLSDCVEKDGFSAVVENDLD


In [3]:
import numpy as np

In [4]:
sampled_seq = model.sample(coords, temperature=1)
print('Sampled sequence:', sampled_seq)

recovery = np.mean([(a==b) for a, b in zip(native_seq, sampled_seq)])
print('Sequence recovery:', recovery)

Sampled sequence: KTLQDWYLSPQNQIQEPEATEDDRLFSLQDMLAMFPVLEPDGLPVWLKFWQSIKRYRMYGPVNPDVDQLRLRMRTAKVVEVTQQRGGRTLTMNVSYEDGGRALWKPAVSFRETAIPPAWRAYEAFSRYRAEVSAYELDNLFDLRSTPPAESRIVNLTTELRDVSSDLALTKTFFVTPKNATCFHGACDHHCDDDHAICGIPNFISGSLEAQLPDDSIAPRWVETSPWAFAWTDEEREDWSRDPHYFEHMSQTPRWRAGQYLLQMANYFIFNFLQGNDDAHTFETFLRYGHRTQLILYDNGYGFGRADYLDKSILTPLRQAAFFRESLWERIQLLSKTETDMQTLLKQVLAKNECYPVLVQPFLAKANTTLSTISSIFAENVKEYGQENVLFKDQF
Sequence recovery: 0.3569620253164557


In [5]:
# Lower sampling temperature typically results in higher sequence recovery but less diversity

sampled_seq = model.sample(coords, temperature=1e-6)
print('Sampled sequence:', sampled_seq)

recovery = np.mean([(a==b) for a, b in zip(native_seq, sampled_seq)])
print('Sequence recovery:', recovery)

Sampled sequence: EVLERWWASPLTQIEDPELSEEDLLFDPEELLALLPEEEEEELPEWLQFWVEIRRRRLYPRQSPWVERLLERLRTAKVREVRQKGGGRSLVLEFSFEDLGRAYFKPKVAELEEETPPEWRYWERLETAQAVVAAYWLDRVLDLRQTPPAAGRRLDLVSELRDVTEDEELLSTFFVTPEERLCFWGRGSFRCSEEHAICGEPEEVEGVLVAALPDERIAPRGLYLSPWRESREEEEEAWWEEDPEFAEHVRRLPPLREGRLLLELARDFVFYFLMGDADAHYYETFERFGLDEFLLIYDTGFGFGRADYLDERILVPLEQGCLLSERLYRRLLLLSEEEYSLERLMEEVLGEDELWPVLSEPFLRQLDRRLERVLEVLEECREEVGEEECLVEEEE
Sequence recovery: 0.43291139240506327


## Conditional sequence log-likelihoods for given backbone coordinates

In [6]:
from inverse_folding import score_sequence
ll_fullseq, ll_withcoord = score_sequence(model, alphabet, coords, native_seq)

print(f'average log-likelihood on entire sequence: {ll_fullseq:.2f} (perplexity {np.exp(-ll_fullseq):.2f})')
print(f'average log-likelihood excluding missing coordinates: {ll_withcoord:.2f} (perplexity {np.exp(-ll_withcoord):.2f})')

average log-likelihood on entire sequence: -1.70 (perplexity 5.48)
average log-likelihood excluding missing coordinates: -1.70 (perplexity 5.48)


## Masking part of the backbone coordinates
To partially mask backbone coordinates, simply set the masked coordinates to np.inf. 

Typically, the sequence perplexity will be higher on masked regions than on unmasked regions.

In [7]:
from inverse_folding import score_sequence
from copy import deepcopy
masked_coords = deepcopy(coords)
masked_coords[:15] = np.inf # mask the first 10 residues
ll_fullseq, ll_withcoord = score_sequence(model, alphabet, masked_coords, native_seq)

print(f'average log-likelihood on entire sequence: {ll_fullseq:.2f} (perplexity {np.exp(-ll_fullseq):.2f})')
print(f'average log-likelihood excluding missing coordinates: {ll_withcoord:.2f} (perplexity {np.exp(-ll_withcoord):.2f})')

average log-likelihood on entire sequence: -1.73 (perplexity 5.65)
average log-likelihood excluding missing coordinates: -1.69 (perplexity 5.41)


## Extract encoder output as structure representation
The encoder output may also be used as a representation for the structure.

For a set of input coordinates with L amino acids, the encoder output will have shape L x 512.

In [8]:
from inverse_folding import get_encoder_output
rep = get_encoder_output(model, alphabet, coords)
len(coords), rep.shape

(395, torch.Size([395, 512]))